## Objective:
Perform EDA on the 

In [1]:
import pandas as pd
import xlwings as xl
import pandas as pd
import plotly.express as px
import numpy as np
import metrics.HRV_Metrics as HRV_Metrics
import pickle
from metrics.SignalPsdPlotter import SignalPsdPlotter
import random
import copy
import metrics.SincPsd as SincPSd
from utils import time_portion_signal
import utils

pd.options.display.float_format = '{:.2f}'.format

# File reading

Accounts for some values being computed within excel.

In [2]:
import xlwings as xw

def df_from_excel(path):
    wb = xw.Book(path)
    wb.app.api.Visible = False
    wb.app.calculate()
    wb.save()
    #wb.close()
    wb.app.quit()
    return pd.read_excel(path)

## Clinical indicators

In [3]:


clin_indic = df_from_excel('actionable_data/Clinical indicators.xlsx')
print(clin_indic.columns)

clin_indic.head(5)

Index(['number', 'gender', 'age', 'height', 'weight', 'admission FBG (mmol/L)',
       'Discharge FBG (mmol/L)', 'HbA1c (%)', 'SBP (mmHg)', 'DBP (mmHg)',
       'WBC (×109/L)', 'N% (%)', 'Hb (g/L)', 'PLT (×109/L)', 'CRP (mg/L)',
       'ALT (U/L)', ' AST (U/L)', 'AST/ALT', 'GGT (U/L)', 'BUN (mmol/L)',
       ' UA (mmol/L)', 'TG (mmol/L)', 'HDL-C (mmol/L)', 'LDL-C (mmol/L)',
       'UMA (mg)', 'UCr (g)', 'UACR (mg/g)', 'Diabetic Complications',
       'Diabetic nephropathy', 'Diabetic retinopathy and cataract',
       'Diabetic peripheral neuropathy',
       'Coronary artery disease and cardiac insufficiency',
       'Lower extremity atherosclerosis or stenosis', 'Carotid plaque'],
      dtype='object')


,number,gender,age,height,weight,admission FBG (mmol/L),Discharge FBG (mmol/L),HbA1c (%),SBP (mmHg),DBP (mmHg),...,UMA (mg),UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque
0,20050628,male,48,170,79.00,10.70,9.20,9.80,133.00,78.00,...,9.41,15.27,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,20052606,male,38,173,90.00,NaN,6.20,NaN,127.00,88.00,...,28.00,14.48,1.93,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,20061729,male,41,185,76.60,13.30,6.70,11.80,140.00,90.00,...,2.93,3.89,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,20092226,male,67,180,64.00,16.50,7.70,NaN,127.00,77.00,...,5.27,5.87,0.90,1.00,0.00,1.00,0.00,0.00,0.00,1.00
4,20092535,male,49,165,65.00,8.80,5.20,10.20,122.00,97.00,...,15.70,15.65,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00


In [42]:
clin_indic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 20050628 to 20125096
Data columns (total 37 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   gender                                             60 non-null     object 
 1   age                                                60 non-null     int64  
 2   height                                             60 non-null     int64  
 3   weight                                             60 non-null     float64
 4   admission FBG (mmol/L)                             52 non-null     float64
 5   Discharge FBG (mmol/L)                             37 non-null     float64
 6   HbA1c (%)                                          40 non-null     float64
 7   SBP (mmHg)                                         58 non-null     float64
 8   DBP (mmHg)                                         58 non-null     float64
 9   WBC 

## Single and confounding complications

In [50]:
clin_indic['Complication'] = clin_indic[['Diabetic nephropathy', 'Diabetic retinopathy and cataract', 'Diabetic peripheral neuropathy', 'Coronary artery disease and cardiac insufficiency', 'Lower extremity atherosclerosis or stenosis', 'Carotid plaque']].sum(axis=1)

count_1 = (clin_indic['Complication'] == 1).sum()
count_0 = (clin_indic['Complication'] == 0).sum()
count_more_than_1 = (clin_indic['Complication'] > 1).sum()

print(f"Patients without complications: {count_0}")
print(f"Patients with a single Complication: {count_1}")
print(f"Patients with confounding complications: {count_more_than_1}")

complication_counts = clin_indic[['Diabetic nephropathy', 'Diabetic retinopathy and cataract', 'Diabetic peripheral neuropathy', 'Coronary artery disease and cardiac insufficiency', 'Lower extremity atherosclerosis or stenosis', 'Carotid plaque']].sum()
display(complication_counts)

Patients without complications: 31
Patients with a single Complication: 17
Patients with confounding complications: 12


Diabetic nephropathy                                 6.00
Diabetic retinopathy and cataract                   16.00
Diabetic peripheral neuropathy                       4.00
Coronary artery disease and cardiac insufficiency    3.00
Lower extremity atherosclerosis or stenosis          4.00
Carotid plaque                                      10.00
dtype: float64

*some patients have multiple complications

In [5]:
clin_indic.describe()

,number,age,height,weight,admission FBG (mmol/L),Discharge FBG (mmol/L),HbA1c (%),SBP (mmHg),DBP (mmHg),WBC (×109/L),...,UMA (mg),UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque
count,60.00,60.00,60.00,60.00,52.00,37.00,40.00,58.00,58.00,56.00,...,49.00,47.00,47.00,58.00,58.00,58.00,58.00,58.00,58.00,58.00
mean,19609115.60,49.53,171.52,77.53,9.05,6.54,8.22,136.03,85.02,7.23,...,56.54,10.38,6.05,0.50,0.10,0.28,0.07,0.05,0.07,0.17
std,502267.04,16.00,6.43,13.94,3.07,1.24,1.67,13.56,10.68,3.99,...,89.41,5.90,10.93,0.50,0.31,0.45,0.26,0.22,0.26,0.38
min,19070921.00,12.00,154.00,50.60,3.30,4.90,5.60,105.00,61.00,3.60,...,2.69,2.33,0.35,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,19098785.25,34.75,169.75,69.00,6.47,5.50,6.70,126.00,78.00,5.30,...,8.10,5.75,0.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,20010826.50,49.00,172.00,76.00,8.55,6.40,8.20,133.50,86.00,6.30,...,14.60,8.77,1.42,0.50,0.00,0.00,0.00,0.00,0.00,0.00
75%,20101530.25,63.00,175.00,81.70,11.07,6.90,9.57,142.75,92.75,7.58,...,51.80,14.88,4.01,1.00,0.00,1.00,0.00,0.00,0.00,0.00
max,20125096.00,77.00,185.00,118.60,16.50,9.30,11.80,169.00,115.00,30.30,...,411.00,26.68,47.84,1.00,1.00,1.00,1.00,1.00,1.00,1.00


# Objective sleep quality

In [6]:
sleep_obj = df_from_excel('actionable_data/Objective sleep quality.xlsx')
print(sleep_obj.columns)
sleep_obj.head(5)

Index(['number', 'gender', 'age', 'height（cm）', 'weight（kg）',
       'subjective sleep quality', 'sleep latency', 'sleep duration',
       'habitual sleep efficiency', 'sleep disturbances', 'sleep medication',
       'daytime dysfunction', 'PSQI score'],
      dtype='object')


,number,gender,age,height（cm）,weight（kg）,subjective sleep quality,sleep latency,sleep duration,habitual sleep efficiency,sleep disturbances,sleep medication,daytime dysfunction,PSQI score
0,20050628,male,48,170.00,79.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
1,20052606,male,38,173.00,90.00,1.00,1.00,0.00,0.00,2.00,0.00,1.00,5.00
2,20061729,male,41,185.00,76.60,1.00,0.00,0.00,0.00,0.00,1.00,1.00,3.00
3,20092226,male,67,160.00,64.00,2.00,2.00,0.00,1.00,2.00,0.00,1.00,8.00
4,20092535,male,49,165.00,68.00,0.00,2.00,0.00,0.00,1.00,0.00,1.00,4.00


In [7]:
sleep_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   number                     59 non-null     object 
 1   gender                     59 non-null     object 
 2   age                        59 non-null     int64  
 3   height（cm）                 58 non-null     float64
 4   weight（kg）                 58 non-null     float64
 5   subjective sleep quality   54 non-null     float64
 6   sleep latency              54 non-null     float64
 7   sleep duration             54 non-null     float64
 8   habitual sleep efficiency  54 non-null     float64
 9   sleep disturbances         54 non-null     float64
 10  sleep medication           54 non-null     float64
 11  daytime dysfunction        54 non-null     float64
 12  PSQI score                 58 non-null     float64
dtypes: float64(10), int64(1), object(2)
memory usage: 6.

In [8]:
sleep_obj.describe()

,age,height（cm）,weight（kg）,subjective sleep quality,sleep latency,sleep duration,habitual sleep efficiency,sleep disturbances,sleep medication,daytime dysfunction,PSQI score
count,59.00,58.00,58.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00,58.00
mean,48.75,170.95,76.71,1.37,1.13,0.59,0.37,1.11,0.37,1.20,5.72
std,16.07,6.66,14.59,0.81,0.91,0.66,0.62,0.46,0.73,0.92,3.32
min,12.00,154.00,45.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,34.00,169.00,68.93,1.00,1.00,0.00,0.00,1.00,0.00,1.00,4.00
50%,49.00,171.50,75.50,1.00,1.00,0.50,0.00,1.00,0.00,1.00,5.00
75%,63.00,175.00,80.00,2.00,1.75,1.00,1.00,1.00,0.75,2.00,8.00
max,77.00,185.00,118.60,3.00,3.00,2.00,3.00,2.00,3.00,3.00,13.00


## Subjective Sleep Quality

In [9]:
sleep_subj = pd.read_excel('actionable_data/Subjective sleep quality.xlsx')
print(sleep_subj.columns)
sleep_subj.head(5)


Index(['number', 'AHI (event per hour)', 'TST (hour)', 'UST (hour)',
       'SST (hour)', 'RST (hour)', 'ALUS (min)', 'ALSS (min)', 'ALRS (min)',
       'USP (%)', 'SSP (%)', 'RSP (%)'],
      dtype='object')


,number,AHI (event per hour),TST (hour),UST (hour),SST (hour),RST (hour),ALUS (min),ALSS (min),ALRS (min),USP (%),SSP (%),RSP (%)
0,19070921,14.70,6.80,2.80,2.10,1.90,7.10,8.80,4.90,0.41,0.31,0.28
1,19072205,20.20,4.20,1.50,2.00,0.80,7.70,13.00,5.10,0.36,0.48,0.19
2,19072214,19.70,11.90,5.50,3.10,3.30,6.40,9.00,4.20,0.46,0.26,0.28
3,19072938,0.40,5.40,1.10,3.20,1.10,4.60,16.00,4.10,0.20,0.59,0.20
4,19072940,0.30,7.30,1.00,4.70,1.60,5.20,22.00,6.50,0.14,0.64,0.22


In [10]:
sleep_subj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number                40 non-null     object 
 1   AHI (event per hour)  40 non-null     float64
 2   TST (hour)            40 non-null     float64
 3   UST (hour)            40 non-null     float64
 4   SST (hour)            40 non-null     float64
 5   RST (hour)            40 non-null     float64
 6   ALUS (min)            40 non-null     float64
 7   ALSS (min)            40 non-null     float64
 8   ALRS (min)            40 non-null     float64
 9   USP (%)               40 non-null     float64
 10  SSP (%)               40 non-null     float64
 11  RSP (%)               40 non-null     float64
dtypes: float64(11), object(1)
memory usage: 4.0+ KB


In [11]:
sleep_subj.describe()

,AHI (event per hour),TST (hour),UST (hour),SST (hour),RST (hour),ALUS (min),ALSS (min),ALRS (min),USP (%),SSP (%),RSP (%)
count,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00
mean,25.15,7.69,3.73,2.44,1.58,10.17,11.03,4.58,0.47,0.32,0.20
std,18.68,1.80,1.62,1.16,0.70,5.55,4.25,1.20,0.18,0.15,0.08
min,0.30,1.54,0.90,0.30,0.20,4.30,3.00,1.08,0.14,0.04,0.03
25%,14.45,7.20,2.72,1.50,1.08,6.35,8.00,4.10,0.35,0.20,0.13
50%,21.20,7.92,3.69,2.29,1.50,8.40,11.00,4.69,0.47,0.32,0.20
75%,36.10,8.83,5.05,3.23,2.02,12.15,13.00,5.15,0.61,0.41,0.27
max,101.70,11.90,6.50,4.70,3.30,27.80,22.00,7.10,0.83,0.65,0.34


# HRV


### Acronyms explanation:
DS - Deep Sleep:
* This is the stage of sleep associated with physical restoration, memory consolidation, and metabolic regulation.
* HRV typically increases in this stage, indicating stronger parasympathetic (vagal) activity.
* There is a decrease in heart rate and an increase in high-frequency (HF) HRV components, reflecting rest and recovery.

REM - Rapid EYE Movement
* The stage where most dreaming occurs, crucial for cognitive function and emotional regulation.
* HRV is more variable in REM, with an interplay of both sympathetic and parasympathetic nervous system influences.
* Studies show reduced vagal tone and increased sympathetic activity compared to Deep Sleep.

RS - Resting State:
* If RS refers to general resting state (or light sleep), it includes stages N1 and N2 of sleep.
* In light sleep (N2), HRV tends to be lower than in Deep Sleep but higher than in REM.
* In awake resting states, HRV varies depending on relaxation levels, with a dominance of parasympathetic control in a fully relaxed state.

In [12]:


# Load the pickled file
peaks = pd.read_pickle('actionable_data/data.pkl')

# Display the dataframe
print(peaks)

labels = [f'{id} DS' for id in peaks.keys()]
DS_RR ={}
for id, patient in peaks.items():
    patient_ds = patient["DS"][0]
    patient_ds= pd.Series(patient_ds, index=patient_ds.cumsum()/1000)
    DS_RR[id]=patient_ds

print(DS_RR)

{'19070921': {'DS': array([[1068, 1016,  968, ..., 1032, 1020, 1044]], dtype=uint16), 'REM': array([[1088, 1112, 1072, ..., 1096, 1064, 1136]], dtype=uint16), 'RS': array([[1204, 1188, 1100, ..., 1004, 1044, 1036]], dtype=uint16)}, '19072205': {'DS': array([[1180, 1048, 1196, ..., 1332, 1220, 1264]], dtype=uint16), 'REM': array([[1128, 1048, 1116, ..., 1132,  964, 1004]], dtype=uint16), 'RS': array([[1068, 1104, 1056, ..., 1064, 1120, 1144]], dtype=uint16)}, '19072214': {'DS': array([[696, 688, 684, ..., 792, 832, 812]], dtype=uint16), 'REM': array([[752, 740, 724, ..., 856, 844, 832]], dtype=uint16), 'RS': array([[720, 696, 700, ..., 800, 796, 784]], dtype=uint16)}, '19072938': {'DS': array([[ 964,  948,  912, ..., 1000, 1048, 1052]], dtype=uint16), 'REM': array([[888, 900, 872, ..., 944, 900, 892]], dtype=uint16), 'RS': array([[1036, 1080,  980, ...,  936,  936,  968]], dtype=uint16)}, '19072940': {'DS': array([[664, 692, 644, ..., 780, 816, 816]], dtype=uint16), 'REM': array([[860, 


fig = px.line(title='Patient Data')

for id, patient in DS_RR.items():
    
    fig.add_scatter(y=patient, mode='lines', name=f'{id} DS')


    fig.update_layout(xaxis_title='Time (s)', yaxis_title='RR period (ms)')

fig.show()


import plotly.figure_factory as ff

fig = ff.create_distplot(list(DS_RR.values()), list(DS_RR.keys()), show_hist=False, show_rug=False, curve_type="kde")
fig.update_layout(title='Patient HRV', xaxis_title='RR period (ms)', yaxis_title='Frequency')

fig.show()

In [13]:
import plotly.graph_objects as go

fig = go.Figure()

# Add box plots for each dataset
for label, ds_data in DS_RR.items():

    fig.add_trace(go.Box(y=ds_data, name=label))

# Update layout
fig.update_layout(
    title='Box Plot of DS Data for Each Patient',
    yaxis_title='RR Period (ms)',
    xaxis_title='Patient ID'
)

fig.show()

In [14]:
# Create new columns for each sleep stage HRV array

print(peaks)

clin_indic['DS_HRV'] = clin_indic['number'].map(lambda x: peaks[str(x)]['DS'][0] if str(x) in peaks else np.nan)
clin_indic['REM_HRV'] = clin_indic['number'].map(lambda x: peaks[str(x)]['REM'][0] if str(x) in peaks else np.nan)
clin_indic['RS_HRV'] = clin_indic['number'].map(lambda x: peaks[str(x)]['RS'][0] if str(x) in peaks else np.nan)


clin_indic.set_index('number', inplace=True)
clin_indic.index.name = 'id'
clin_indic.head(2)

{'19070921': {'DS': array([[1068, 1016,  968, ..., 1032, 1020, 1044]], dtype=uint16), 'REM': array([[1088, 1112, 1072, ..., 1096, 1064, 1136]], dtype=uint16), 'RS': array([[1204, 1188, 1100, ..., 1004, 1044, 1036]], dtype=uint16)}, '19072205': {'DS': array([[1180, 1048, 1196, ..., 1332, 1220, 1264]], dtype=uint16), 'REM': array([[1128, 1048, 1116, ..., 1132,  964, 1004]], dtype=uint16), 'RS': array([[1068, 1104, 1056, ..., 1064, 1120, 1144]], dtype=uint16)}, '19072214': {'DS': array([[696, 688, 684, ..., 792, 832, 812]], dtype=uint16), 'REM': array([[752, 740, 724, ..., 856, 844, 832]], dtype=uint16), 'RS': array([[720, 696, 700, ..., 800, 796, 784]], dtype=uint16)}, '19072938': {'DS': array([[ 964,  948,  912, ..., 1000, 1048, 1052]], dtype=uint16), 'REM': array([[888, 900, 872, ..., 944, 900, 892]], dtype=uint16), 'RS': array([[1036, 1080,  980, ...,  936,  936,  968]], dtype=uint16)}, '19072940': {'DS': array([[664, 692, 644, ..., 780, 816, 816]], dtype=uint16), 'REM': array([[860, 

,gender,age,height,weight,admission FBG (mmol/L),Discharge FBG (mmol/L),HbA1c (%),SBP (mmHg),DBP (mmHg),WBC (×109/L),...,Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque,DS_HRV,REM_HRV,RS_HRV
id,,,,,,,,,,,,,,,,,,,,,
20050628,male,48,170,79.00,10.70,9.20,9.80,133.00,78.00,6.10,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"[656, 648, 652, 656, 664, 672, 680, 676, 696, ...","[888, 904, 916, 888, 892, 908, 900, 884, 892, ...","[872, 872, 860, 860, 872, 852, 844, 860, 848, ..."
20052606,male,38,173,90.00,NaN,6.20,NaN,127.00,88.00,6.30,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"[676, 684, 692, 692, 680, 688, 696, 700, 692, ...","[688, 688, 692, 700, 704, 696, 700, 704, 700, ...","[796, 772, 784, 800, 800, 780, 784, 800, 784, ..."


# Power Spectral Density
## Expected shape

From: [Fluctuations in autonomic nervous activity during sleep displayed by power spectrum analysis of heart rate variability](https://www.neurology.org/doi/abs/10.1212/WNL.45.6.1183)
![Baharav_1995.png](supporting_images/Baharav_1995.png){: width="200px" }

## psd attempt

The following PSD signals were calculated using the Welch Method

In [37]:
patient_id = 20052606

clin_indic.loc[patient_id]

gender                                                                                            male
age                                                                                                 38
height                                                                                             173
weight                                                                                           90.00
admission FBG (mmol/L)                                                                             NaN
Discharge FBG (mmol/L)                                                                            6.20
HbA1c (%)                                                                                          NaN
SBP (mmHg)                                                                                      127.00
DBP (mmHg)                                                                                       88.00
WBC (×109/L)                                                             

In [ ]:
# Get the DS data for patient 19072205


trial = clin_indic.loc[patient_id, 'DS_HRV']



plotter = SignalPsdPlotter(title = f"HRV Signal and PSD plots for patient id {patient_id}")


for i, five_min in enumerate(time_portion_signal(trial)):
    color = f'#{random.randint(0, 0xFFFFFF):06x}'
    plotter.calc_and_plot(five_min, str(i), color)

plotter.show()

plotter.fig.write_html(f'html_plots/one_patient_psd_plot_{patient_id}.html')

# Metrics

In [ ]:
metrics = utils.patients_metrics(trial)

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio
0,0,300,21.72,15.63,0.71,84.67,0.00,0.00,0.38,0.03,2.41,0.11,57.97,0.34,0.04
1,300,600,31.89,16.03,0.92,86.76,0.00,0.00,1.70,0.03,3.16,0.06,37.41,0.34,0.08
2,600,900,41.13,14.88,0.42,95.70,0.00,0.00,1.58,0.03,4.81,0.05,12.86,0.38,0.37
3,900,1200,40.32,20.32,2.10,95.58,0.00,0.00,0.45,0.03,13.63,0.06,25.84,0.37,0.53
4,1200,1500,30.67,11.91,0.23,88.30,0.00,0.00,0.47,0.03,6.09,0.11,11.11,0.31,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,16800,17100,70.03,43.48,20.06,67.13,0.00,0.00,5.23,0.03,126.36,0.12,304.77,0.20,0.41
57,17100,17400,87.36,40.27,17.56,67.41,0.00,0.00,9.95,0.03,76.29,0.13,236.66,0.18,0.32
58,17400,17700,86.07,38.03,14.71,68.22,0.00,0.00,16.38,0.03,110.09,0.13,223.55,0.17,0.49
59,17700,18000,114.32,25.77,6.09,69.19,0.00,0.00,15.07,0.03,53.24,0.05,74.20,0.23,0.72


In [17]:
metrics.head(3)

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio
count,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00
mean,9000.00,9300.00,85.01,35.84,11.97,74.03,0.00,0.00,16.26,0.03,74.22,0.08,239.01,0.27,0.35
std,5325.88,5325.88,34.31,14.13,8.98,8.43,0.00,0.00,14.33,0.01,57.42,0.03,195.53,0.06,0.18
min,0.00,300.00,21.72,11.91,0.00,64.74,0.00,0.00,0.27,0.00,2.41,0.04,8.01,0.17,0.04
25%,4500.00,4800.00,49.58,21.65,2.27,68.43,0.00,0.00,2.40,0.03,16.40,0.06,65.05,0.23,0.20
50%,9000.00,9300.00,88.80,38.80,13.95,69.95,0.00,0.00,15.24,0.03,76.57,0.07,223.55,0.25,0.34
75%,13500.00,13800.00,110.75,47.92,18.82,77.39,0.00,0.00,21.71,0.03,110.69,0.11,335.14,0.32,0.44
max,18000.00,18300.00,155.33,62.87,31.67,97.05,0.00,0.00,59.06,0.04,281.16,0.15,767.71,0.39,0.84


In [24]:
metrics.describe()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio
count,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00,61.00
mean,9000.00,9300.00,85.01,35.84,11.97,74.03,0.00,0.00,16.26,0.03,74.22,0.08,239.01,0.27,0.35
std,5325.88,5325.88,34.31,14.13,8.98,8.43,0.00,0.00,14.33,0.01,57.42,0.03,195.53,0.06,0.18
min,0.00,300.00,21.72,11.91,0.00,64.74,0.00,0.00,0.27,0.00,2.41,0.04,8.01,0.17,0.04
25%,4500.00,4800.00,49.58,21.65,2.27,68.43,0.00,0.00,2.40,0.03,16.40,0.06,65.05,0.23,0.20
50%,9000.00,9300.00,88.80,38.80,13.95,69.95,0.00,0.00,15.24,0.03,76.57,0.07,223.55,0.25,0.34
75%,13500.00,13800.00,110.75,47.92,18.82,77.39,0.00,0.00,21.71,0.03,110.69,0.11,335.14,0.32,0.44
max,18000.00,18300.00,155.33,62.87,31.67,97.05,0.00,0.00,59.06,0.04,281.16,0.15,767.71,0.39,0.84


In [73]:
import matplotlib.pyplot as plt


def plot_beeswarm(metrics_df):
    # Exclude 't_start' and 't_end' columns
    #metrics_df = metrics_df.drop(columns=['t_start', 't_end'])
    
    # Create a figure for each column
    for col in metrics_df.columns[2:]:
        fig = px.violin(metrics_df, x=col, box=True, points="all", title=f"<b>{col} distribution</b>")

        fig.update_layout(title={'text': col, 'x': 0.5})
        fig.update_layout(xaxis_title='Value')
        
        fig.update_layout(width=800, height=300)

        fig.update_traces(customdata=metrics_df['t_start'])
        fig.update_traces(hovertemplate='Value: %{x}<br>t_start: %{customdata}')
        

        fig.update_layout(
    margin=dict(l=10,r=10,b=10,t=40),
   # paper_bgcolor="Black"
    )

        fig.show()

# Call the function to plot the metrics dataframe
plot_beeswarm(metrics)

# Population wide analysis

In [76]:
full_data = clin_indic
full_data.head(2)

,gender,age,height,weight,admission FBG (mmol/L),Discharge FBG (mmol/L),HbA1c (%),SBP (mmHg),DBP (mmHg),WBC (×109/L),...,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque,DS_HRV,REM_HRV,RS_HRV,Complication
id,,,,,,,,,,,,,,,,,,,,,
20050628,male,48,170,79.00,10.70,9.20,9.80,133.00,78.00,6.10,...,0.00,0.00,0.00,0.00,0.00,0.00,"[656, 648, 652, 656, 664, 672, 680, 676, 696, ...","[888, 904, 916, 888, 892, 908, 900, 884, 892, ...","[872, 872, 860, 860, 872, 852, 844, 860, 848, ...",0.00
20052606,male,38,173,90.00,NaN,6.20,NaN,127.00,88.00,6.30,...,0.00,0.00,0.00,0.00,0.00,0.00,"[676, 684, 692, 692, 680, 688, 696, 700, 692, ...","[688, 688, 692, 700, 704, 696, 700, 704, 700, ...","[796, 772, 784, 800, 800, 780, 784, 800, 784, ...",0.00


In [78]:
# Function to derive metrics for each HRV type
def derive_metrics(hrv_data):
    metrics_list = []
    for id, data in hrv_data.items():
        metrics = utils.patients_metrics(data)
        metrics['id'] = id
        metrics_list.append(metrics)
    return pd.concat(metrics_list, ignore_index=True)

# Derive metrics for DS_HRV, REM_HRV, and RS_HRV
ds_hrv_metrics = derive_metrics({id: peaks[id]['DS'][0] for id in peaks})
ds_hrv_metrics.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio,id
0,0,300,38.71,45.25,35.45,53.88,0.00,0.00,1.71,0.03,16.41,0.05,626.48,0.25,0.03,19070921
1,300,600,110.35,42.14,21.86,62.38,0.00,0.00,39.45,0.04,54.26,0.06,589.95,0.25,0.09,19070921
2,600,900,43.16,33.64,14.53,58.01,0.00,0.00,1.69,0.02,16.59,0.07,248.37,0.24,0.07,19070921
3,900,1200,91.95,39.51,24.91,57.92,0.00,0.00,9.35,0.03,24.38,0.07,395.81,0.25,0.06,19070921
4,1200,1500,43.78,41.08,26.06,57.07,0.00,0.00,2.55,0.04,35.26,0.07,466.23,0.24,0.08,19070921


In [79]:
rem_hrv_metrics = derive_metrics({id: peaks[id]['REM'][0] for id in peaks})
rem_hrv_metrics .head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio,id
0,0,300,130.04,44.41,28.52,59.74,0.00,0.00,7.69,0.01,37.21,0.07,597.43,0.26,0.06,19070921
1,300,600,77.51,33.93,16.50,59.69,0.00,0.00,3.94,0.03,24.43,0.07,271.55,0.25,0.09,19070921
2,600,900,81.00,24.91,5.86,64.88,0.00,0.00,8.30,0.02,15.93,0.06,162.84,0.26,0.10,19070921
3,900,1200,113.58,40.02,11.25,62.58,0.00,0.00,15.66,0.04,61.16,0.07,404.54,0.32,0.15,19070921
4,1200,1500,47.22,37.95,20.42,56.84,0.00,0.00,4.92,0.02,22.15,0.05,395.80,0.22,0.06,19070921


In [80]:
rs_hrv_metrics = derive_metrics({id: peaks[id]['RS'][0] for id in peaks})
rs_hrv_metrics.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio,id
0,0,300,43.24,55.08,44.32,53.00,0.00,0.00,0.81,0.03,18.20,0.08,883.39,0.23,0.02,19070921
1,300,600,42.83,55.62,49.24,52.64,0.00,0.00,1.29,0.04,14.94,0.10,983.89,0.25,0.02,19070921
2,600,900,44.61,55.80,46.01,52.80,0.00,0.00,1.38,0.04,13.99,0.08,1128.47,0.25,0.01,19070921
3,900,1200,38.57,49.74,39.02,53.07,0.00,0.00,0.65,0.04,26.76,0.06,750.02,0.24,0.04,19070921
4,1200,1500,41.12,46.85,35.32,53.87,0.00,0.00,1.14,0.04,16.56,0.10,715.65,0.25,0.02,19070921


In [82]:
ds_hrv_metrics['sleep_type'] = 'DS'
rem_hrv_metrics['sleep_type'] = 'REM'
rs_hrv_metrics['sleep_type'] = 'RS'

ds_hrv_metrics.head()
rem_hrv_metrics.head()
rs_hrv_metrics.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,LF Power,LF Peak Frequency,HF Power,HF Peak Frequency,LF/HF Ratio,id,sleep_type
0,0,300,43.24,55.08,44.32,53.00,0.00,0.00,0.81,0.03,18.20,0.08,883.39,0.23,0.02,19070921,RS
1,300,600,42.83,55.62,49.24,52.64,0.00,0.00,1.29,0.04,14.94,0.10,983.89,0.25,0.02,19070921,RS
2,600,900,44.61,55.80,46.01,52.80,0.00,0.00,1.38,0.04,13.99,0.08,1128.47,0.25,0.01,19070921,RS
3,900,1200,38.57,49.74,39.02,53.07,0.00,0.00,0.65,0.04,26.76,0.06,750.02,0.24,0.04,19070921,RS
4,1200,1500,41.12,46.85,35.32,53.87,0.00,0.00,1.14,0.04,16.56,0.10,715.65,0.25,0.02,19070921,RS


In [87]:
# Concatenate the dataframes
combined_metrics = pd.concat([ds_hrv_metrics, rem_hrv_metrics, rs_hrv_metrics], ignore_index=True)
combined_metrics['id'] = combined_metrics['id'].astype(str)
# Display the combined dataframe
combined_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3401 entries, 0 to 3400
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   t_start             3401 non-null   int64  
 1   t_end               3401 non-null   int64  
 2   SDRR                3401 non-null   float64
 3   RMSSD               3401 non-null   float64
 4   pNN50 (%)           3401 non-null   float64
 5   Mean HR (bpm)       3401 non-null   float64
 6   ULF Power           3401 non-null   float64
 7   ULF Peak Frequency  3401 non-null   float64
 8   VLF Power           3401 non-null   float64
 9   VLF Peak Frequency  3401 non-null   float64
 10  LF Power            3401 non-null   float64
 11  LF Peak Frequency   3401 non-null   float64
 12  HF Power            3401 non-null   float64
 13  HF Peak Frequency   3401 non-null   float64
 14  LF/HF Ratio         3401 non-null   float64
 15  id                  3401 non-null   object 
 16  sleep_

In [89]:
clin_indic = clin_indic.drop(columns=['DS_HRV', 'REM_HRV', 'RS_HRV'])

clin_indic.index = clin_indic.index.astype(str)
clin_indic.head()


,gender,age,height,weight,admission FBG (mmol/L),Discharge FBG (mmol/L),HbA1c (%),SBP (mmHg),DBP (mmHg),WBC (×109/L),...,UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque,Complication
id,,,,,,,,,,,,,,,,,,,,,
20050628,male,48,170,79.00,10.70,9.20,9.80,133.00,78.00,6.10,...,15.27,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20052606,male,38,173,90.00,NaN,6.20,NaN,127.00,88.00,6.30,...,14.48,1.93,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20061729,male,41,185,76.60,13.30,6.70,11.80,140.00,90.00,5.00,...,3.89,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20092226,male,67,180,64.00,16.50,7.70,NaN,127.00,77.00,8.50,...,5.87,0.90,1.00,0.00,1.00,0.00,0.00,0.00,1.00,2.00
20092535,male,49,165,65.00,8.80,5.20,10.20,122.00,97.00,5.30,...,15.65,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00


In [90]:
# Join combined_metrics and clin_indic on 'id'
merged_data = combined_metrics.merge(clin_indic, left_on='id', right_index=True, how='inner')

# Display the merged dataframe
merged_data.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque,Complication
0,0,300,38.71,45.25,35.45,53.88,0.00,0.00,1.71,0.03,...,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,300,600,110.35,42.14,21.86,62.38,0.00,0.00,39.45,0.04,...,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,600,900,43.16,33.64,14.53,58.01,0.00,0.00,1.69,0.02,...,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,900,1200,91.95,39.51,24.91,57.92,0.00,0.00,9.35,0.03,...,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1200,1500,43.78,41.08,26.06,57.07,0.00,0.00,2.55,0.04,...,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [91]:
# Save as pickle
merged_data.to_pickle('article_derived_HRV_data.pkl')

# Save as CSV
merged_data.to_csv('article_derived_HRV_data.csv', index=False)